# Procedimiento para Catalogar Cáncer Luminal A y B

Este archivo contiene el proceso de limpieza, exploración y análisis de la base de datos del proyecto de de cáncer de mama en base a la información de los diagnosticos de diversos pacientes registrados por la Secretaria de Salud y la información de utilidad proporcionada por el Ingeniero Jesús Esteban Cienfuegos Zurita.

Utilizamos la metodología CRISSP para la comprensión del negocio, la comprensión de los datos reales, y la preparación de los datos. Esto nos permite comenzar a formar una estructuración de los datos y de lo que conocemos de ellos para observar a fondo que información nos pueden proporcionar; la información que buscamos nos permitirá desarrollar un modelo predictivo cómo parte de las siguientes fases del proyecto.



---

##### 1. Exploración y Limpieza de Datos

In [108]:
"""
Pandas es una biblioteca de Python especializada en la manipulación y análisis de datos.
Nos proporciona estructuras como el Dataframe que facilita la limpieza, transformación, exploración y visualización de datos.

Para este proyecto, pandas permite organizar la información de los pacientes, realizar filtrados, 
conversiones de tipos de datos y preparar la base para el análisis profundo y modelado predictivo.
"""

import pandas as pd

# Leemos la Base de Datos y mostramos todas las columnas para tener una mejor idea de su estructura en este mismo archivo

df = pd.read_csv('./resources/2025_08_27 BD Proy Salud.csv', encoding='latin1')#.set_index('ID')


# Deseamos ver todas las columnas para revisar bien cada una de nuestras variables

pd.set_option('display.max_columns', None)

# Mostramos las primeras 4 filas para ver un resumen rápido que nos ayude a entender la estructura de los datos



df.head(4)


,ID,Fecha de Registro,Fecha de Nacimiento,Fecha de Dx biopsia,Edad al Dx (años),Fecha 1° consulta OM CECAN,Peso al Dx,Talla al Dx,IMC,Edo Menopausia Dx,Preservación de la fertilidad,Prueba Genetica,Resultado panel genético,Otra mutación,Significado,CM bilateral,CM asociado al embarazo,EC,T,N,M,Tipo histológico,Otro,Grado,ER,ER_%,PR,PR_%,HER2 / NEU,HER2 IHQ +++,FISH/SISH,Ki67,Ki67_%,ILV,TILs,TILs_%,Patología Rev CECAN,Tx Neoadyuvante,Tx Neoadyuvante CECAN,Quimioterapia Neoadyuvante,Esquema QT Neoadyuvante,AntiHER2 Neoadyuvante,Esquema AntiHER2 Neoadyuvante,Inmunoterapia Neoadyuvante,Comentarios Tx Neoadyuvante,Respuesta patológica completa,ypT,ypN,RBC,Fecha de Cx,Cx CECAN,Tipo Cx mama,Manejo Axila,Requirio 2° Cx de mama,Cual,RT Adyuvante,Tipo RT ady,Analógos GnRH como protector ovarico,Tx Adyuvante,Tx Adyuvante CECAN,Quimioterapia Adyuvante,Esquema QT Adyuvante,Capecitabina,AntiHER2 Adyuvante,Esquema AntiHER2 Adyuvante,Inmunoterapia Adyuvante,Olaparib Adyuvante,Abemaciclib Adyuvante,Adyuvancia endócrina,Adyuvancia endócrina CECAN,Tipo Tx antihormonal (2 primeros años),Switch hormonal (despues de 2 años),Supresión ovarica,Fecha Inicio adyuvancia endócrina,Fin de adyuvancia endócrina,Fecha fin adyuvancia endócrina,Adyuvancia extendida,Ac. Zoledrónico/Denosumab adyuvante,Recurrencia Local,Fecha 1° recurrencia local,SLE (meses),No. de recurrencias locales,Recurrencia sistémica,Fecha 1° recurrencia sistémica,SLEm (meses),Biopsia de 1° recurrencia (local o sistémica),ER recurrencia,ER %,PR recurrencia,PR %,HER2 IHQ recurrencia,HER2 IHQ +++.1,iCDK 4/6 enfermedad metastásica,Tipo iCDK 4/6,Inmunoterapia enfermedad metastásica,Tipo inmunoterapia,iPARP enfermedad metastásica,Estado en ultimo seguimiento,Fecha de ultimo seguimiento,Fecha de muerte,Seguimiento (meses),SG (meses),Muerte por Ca de mama,Ooforectomía,Mastecomía contralateral 2° tiempo,Embarazo post CM,2° Primario,Sitio,Protocolo de investigación,Nombre de protocolo,COMENTARIOS
0,1003-24,1/5/2024,9/28/1977,11/24/2023,46,1/5/2024,76,1.64,28.26,Pre,No aplica,No,No se realizo,NaN,No se realizo,No,No,IIA,1,1,0,CDI,NaN,2,Positivo,10,Positivo,100,Negativo,1+,No aplica,Si,5,No,Desconocido,No aplica,No,No,No,No aplica,No aplica,No aplica,No aplica,No aplica,NaN,No aplica,No aplica,No aplica,No aplica,12/10/2023,No,Mastectomia radical,Disección axilar,No,No aplica,Si,15 Fx,No aplica,Si,Si,Si,AC-T,No,No,No aplica,No,No,No,Si,Si,Tamoxifeno,No,No aplica,8/22/2024,No,No aplica,Por definir,No,No,No aplica,NaN,0,No,No aplica,NaN,No aplica,No aplica,No aplica,No aplica,No aplica,No aplica,No aplica,No aplica,No aplica,No aplica,NaN,No aplica,Viva sin enfermedad,8/22/2024,No aplica,7,0,No,No,No,No,No,NaN,No,NaN,--
1,1008-18,1/25/2018,4/1/1961,1/12/2018,56,1/25/2018,102,1.61,39.35,Post,No aplica,No,No se realizo,No aplica,No se realizo,No,No,IIA,2,0,0,CDI,No aplica,3,Positivo,10,Positivo,5,Negativo,1+,No aplica,No,No aplica,Si,Desconocido,No aplica,No,No,No aplica,No aplica,No aplica,No aplica,No aplica,No aplica,No aplica,No aplica,No aplica,No aplica,No aplica,2/12/2018,Si,Mastectomia total,Ganglio centinela,No,No aplica,No,No aplica,No aplica,Si,Si,Si,AC,No,No aplica,No aplica,No aplica,No,No,Si,Si,Exemestano,No,No aplica,12/7/2018,No,No aplica,No,No,No,No aplica,No aplica,0,Si,2/6/2020,23,Si,Positivo,15,Positivo,0,Negativo,0+,Si,Abemaciclib,No,No aplica,No,Viva con enfermedad,9/28/2023,No aplica,68,68,No,No,No,No,No,No aplica,No,No aplica,Metastasis en pulmon confirmado por TAC
2,1008-20,12/3/2019,2/10/1979,1/4/2020,40,1/17/2020,47,1.58,18.83,Pre,No,No,No se realizo,No aplica,No se realizo,Si,No,No aplica,No aplica,No aplica,No aplica,No aplica,No aplica,No aplica,No aplica,No aplica,No aplica,No aplica,No aplica,No aplica,No aplica,No aplica,No aplica,No aplica,No aplica,No aplica,Si,No,No aplica,No aplica,No aplica,No aplica,No aplica,No aplica,Tambien se comenta tratamiento neo ad con tamo...,No aplica,No aplica,No aplica,No aplica,9/10/2020,Si,Mastectomia bilateral,Disección axilar,No,No 

##### Observaciones iniciales:

- Tenemos muchas variables con nombres muy parecidos, por lo que debemos profundizar en la exploración de los mismos.

- Podemos facilitar limpieza simplificando algunos datos (e.g. quitar espacios, quitar simbolos redundantes), que al final podríamos devolver debido a que no conocemos como afectaría esta alteración a quien nos expidió esta información.

- Podemos notar que existen valores nulos y duplicados en algunas columnas clave, lo que requerirá una limpieza cuidadosa para asegurar la calidad del análisis.

- Notamos reportes de recurrencias en diagnósticos.

---
##### En base a las variables que necesitamos para la catalogación y la exploración inicial de las columnas proporcionadas y su significado...

- Comenzaremos con la reducción de la base de datos para trabajar con los datos que nos corresponden.

- A partir de esta nueva base de datos, limpiamos más a fondo, optimizamos algunos datos para facilitar la manipulación.

- Reconocemos que no sabemos todo, por lo que justificaremos nuestras modificaciones y las comentaremos con nuestro profesor y socio-formador.


---
Antes de elegir los datos, recordemos que una de las observaciones era que hay recurrencia en los diagnosticos. Esto puede ser de importancia para tener una comprensión más amplia del comportamiento de las variables involucradas con el paso del tiempo de una consulta a otra.

In [109]:
# Consultamos cuantas personas realmente tuvieron un segundo diagnóstico registrado en base a las fechas
# de la columna 'Fecha 1° recurrencia sistémica'

if 'Fecha 1° recurrencia sistémica' in df.columns:
    numUnique = df['Fecha 1° recurrencia sistémica'].nunique(dropna=True)
    print(f"Cantidad de valores únicos en 'Fecha 1° recurrencia sistémica': {numUnique}") #Mostramos la cantidad de valores únicos

countNoAplica = df['Fecha 1° recurrencia sistémica'].astype(str).str.strip().str.lower().eq('no aplica').sum() #Mostramos los valores que digan "No Aplica" sin importar mayusculas ni minusculas
print(f"Cantidad de datos 'No Aplica' en 'Fecha 1° recurrencia sistémica': {countNoAplica}")

Cantidad de valores únicos en 'Fecha 1° recurrencia sistémica': 123
Cantidad de datos 'No Aplica' en 'Fecha 1° recurrencia sistémica': 1232


Como se puede observar, hay hasta 123 datos distintos. Al ver cuantos datos coinciden con el indicador "No Aplica", tenemos que la gran mayoría de los registros en esta columna son de personas que no han tenido una primera recurrencia para otro diagnóstico.

Debido a esto, SI se va a omitir la consideración de estos datos para nuestra base de datos reducida.

In [110]:
"""
NOTA: Las justificaciones seran comentadas encima de las lineas de código, 
      mientras que explicaciones técnicas sobre la programación serán comentadas 
      al lado de las lineas SOLO cuando sea necesario.
"""

# Nos deshacemos de las columnas que no aportan valor y tienen nombres similares a otras que si usaremos
# Estas columnas nos están diciendo simplemente el porcentaje de recurrencia de Estrógeno y Prostageno, no el nivel real de estos en el paciente.

df = df.drop(columns=['ER %', 'PR %']) 

# Renombramos columnas para agregar "_" entre espacios

df.columns = df.columns.str.replace(' ', '_') 

# Pondremos en mayuscula la primera letra de cada palabra como mera formalidad

df.columns = df.columns.str.title()

# Mostramos las columnas/variables únicas que tenemos en la base de datos

unique_columns = df.columns.tolist() # Convertimos las variables en una lista para mostrarlas facilmente

print("Numero de columnas/variables: \n", len(unique_columns), "\n") # Imprimimos cuantas columnas tenemos

# Mostramos las columnas y sus nombre en una lista para observar si hay errores o detalles a mejorar una vez realizados los cambios

print("Columnas identificadas: \n", unique_columns) 

Numero de columnas/variables: 
 109 

Columnas identificadas: 
 ['Id', 'Fecha_De_Registro', 'Fecha_De_Nacimiento', 'Fecha_De_Dx_Biopsia', 'Edad_Al_Dx_(Años)', 'Fecha_1°_Consulta_Om_Cecan', 'Peso_Al_Dx', 'Talla_Al_Dx', 'Imc', 'Edo_Menopausia_Dx', 'Preservación_De_La_Fertilidad', 'Prueba_Genetica', 'Resultado_Panel_Genético', 'Otra_Mutación', 'Significado', 'Cm_Bilateral', 'Cm_Asociado_Al_Embarazo', 'Ec', 'T', 'N', 'M', 'Tipo_Histológico', 'Otro', 'Grado', 'Er', 'Er_%', 'Pr', 'Pr_%', 'Her2_/_Neu', 'Her2_Ihq_+++', 'Fish/Sish', 'Ki67', 'Ki67_%', 'Ilv', 'Tils', 'Tils_%', 'Patología_Rev_Cecan', 'Tx_Neoadyuvante', 'Tx_Neoadyuvante_Cecan', 'Quimioterapia_Neoadyuvante', 'Esquema_Qt_Neoadyuvante', 'Antiher2_Neoadyuvante', 'Esquema_Antiher2_Neoadyuvante', 'Inmunoterapia_Neoadyuvante', 'Comentarios_Tx_Neoadyuvante', 'Respuesta_Patológica_Completa', 'Ypt', 'Ypn', 'Rbc', 'Fecha_De_Cx', 'Cx_Cecan', 'Tipo_Cx_Mama', 'Manejo_Axila', 'Requirio_2°_Cx_De_Mama', 'Cual', 'Rt_Adyuvante', 'Tipo_Rt_Ady', 'Analó

En esta acción inicial: 
1. Nos deshicimos de los espacios en los nombres de las columnas con el fin de simplificar su manipulación.

2. Eliminamos las variables de "PR %" y "ER %" de la recurrencia, para que no se nos cruzen ni sean confundidas con las de la consulta inicial.
 
3. Retornamos una lista de todas las columnas para identificar los cambios que hicimos e identificar errores o mejoras.


---
##### Creamos una base de datos reducida con las columnas que corresponden al objetivo de nuestro análisis.

Seleccionamos datos de interés para el análisis de cáncer luminal, además de datos demográficos básicos (con la intención de obtención de nuevos datos para sacar inferencias y tendencias a futuro).


In [111]:
# Seleccionamos datos de interés para el análisis de cáncer luminal, además de datos demográficos básicos


luminal_columns = [
    'Id', 'Fecha_De_Nacimiento', 'Fecha_De_Dx_Biopsia', 'Edad_Al_Dx_(Años)',
    'Fecha_1°_Consulta_Om_Cecan', 'Imc', 'Er_%', 'Pr_%',
    'Her2_Ihq_+++','Fish/Sish', 'Ki67_%', 'Grado'
]


# Nueva DB con columnas de interés, la llamamos luminalDf, para denotar "Luminal Dataframe"

luminalDf = df[luminal_columns]

¿Porqué mantener demográficos y de consulta?

1. Nos permiten expandir nuestras conclusiones al añadir tendencias relacionadas con la edad, tiempos de atención entre fechas, etc.

2. El IMC nos podría servir para identificar repercusiones de fenómenos cómo la obesidad o peso en el desarrollo de este cáncer.
---

In [112]:
# Mostramos todas las columnas y los primeros registros

pd.set_option('display.max_columns', None) #Para ver cada una de las columnas
luminalDf.head()


,Id,Fecha_De_Nacimiento,Fecha_De_Dx_Biopsia,Edad_Al_Dx_(Años),Fecha_1°_Consulta_Om_Cecan,Imc,Er_%,Pr_%,Her2_Ihq_+++,Fish/Sish,Ki67_%,Grado
0,1003-24,9/28/1977,11/24/2023,46,1/5/2024,28.26,10,100,1+,No aplica,5,2
1,1008-18,4/1/1961,1/12/2018,56,1/25/2018,39.35,10,5,1+,No aplica,No aplica,3
2,1008-20,2/10/1979,1/4/2020,40,1/17/2020,18.83,No aplica,No aplica,No aplica,No aplica,No aplica,No aplica
3,1009-19,1/23/1975,11/6/2018,43,1/14/2019,19.91,0,60,1+,No aplica,20,1
4,1009-23,11/30/1974,12/30/2022,48,1/25/2023,35.65,80,90,0+,No aplica,No aplica,2


In [113]:
# Mostramos la cantidad de registros

print("Cantidad de registros:", luminalDf.shape[0]) # shape[0] nos da el número de registros en el DataFrame

Cantidad de registros: 1366


Tenemos 1366 registros, ahora empezamos la limpieza buscando IDs vacíos o duplicados

In [114]:
#Limpiamos IDs repetidos y nulos, nos quedamos con los primeros repetidos


# Identificamos los IDs duplicados
duplicated_ids = luminalDf[luminalDf.duplicated(subset=['Id'], keep=False)]



# Comparamos si los duplicados son exactamente iguales en todas las columnas
comparison = duplicated_ids.groupby('Id').apply(lambda x: x.nunique() > 1)
print("\n¿Los duplicados tienen columnas con datos distintos?")
print(comparison)

# Eliminamos solo los duplicados que son exactamente iguales en todas las columnas
ids_to_drop = comparison.index[~comparison.any(axis=1)]
luminalDf = luminalDf[~luminalDf['Id'].isin(ids_to_drop) | ~luminalDf.duplicated(subset=luminalDf.columns, keep='first')]
luminalDf = luminalDf.drop_duplicates(subset=['Id'], keep='first')
luminalDf = luminalDf[luminalDf['Id'].notna()]

#Mostramos la cantidad de registros después de la limpieza
print("Cantidad de registros después de la limpieza:", luminalDf.shape[0])


¿Los duplicados tienen columnas con datos distintos?
            Id  Fecha_De_Nacimiento  Fecha_De_Dx_Biopsia  Edad_Al_Dx_(Años)  \
Id                                                                            
1012-23  False                False                False              False   
1245-23  False                False                False              False   
1358-24  False                False                 True              False   
1402-23  False                False                False              False   
1413-23  False                False                False              False   
1468-23  False                False                False              False   
1647-20  False                False                False              False   
1656-18  False                False                False              False   
1746-23  False                False                False              False   
1755-23  False                False                False              False  

C:\Users\gilin\AppData\Local\Temp\ipykernel_11376\243991274.py:10: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  comparison = duplicated_ids.groupby('Id').apply(lambda x: x.nunique() > 1)


In [135]:
# Mostramos los datos duplicados que tienen diferencias en alguna columna, solo mostrar donde esta la diferencia
diff_dupes = duplicated_ids.groupby('Id').filter(lambda x: x.nunique().gt(1).any())
print(diff_dupes[diff_dupes.columns[diff_dupes.nunique() > 1]])

          Id Fecha_De_Nacimiento Fecha_De_Dx_Biopsia  Edad_Al_Dx_(Años)  \
285  1358-24          11/28/1959           3/14/2024                 64   
286  1358-24          11/28/1959           2/14/2024                 64   
507  1656-18           5/27/1972            4/1/2018                 45   
508  1656-18           5/27/1972            4/1/2018                 45   
662  1885-22            9/5/1956           7/13/2022                 65   
663  1885-22            9/5/1956           7/13/2022                 65   

    Fecha_1°_Consulta_Om_Cecan    Imc Er_% Pr_% Her2_Ihq_+++     Ki67_% Grado  
285                   4/9/2024  24.53  100  100           1+  No aplica     3  
286                   4/9/2024  24.53  100  100           1+  No aplica     3  
507                   5/7/2018  41.12    0   40           1+          5     2  
508                  7/26/2018  41.12    0   40           1+          5     2  
662                   7/8/2022  35.34   60    0           0+         20   

Al menos 15 registros estaban duplicados o eran nulos

---
Analizamos los tipos de datos de las columnas

In [115]:
print("Tipos de datos de cada columna:")
print(luminalDf.dtypes)

Tipos de datos de cada columna:
Id                             object
Fecha_De_Nacimiento            object
Fecha_De_Dx_Biopsia            object
Edad_Al_Dx_(Años)               int64
Fecha_1°_Consulta_Om_Cecan     object
Imc                           float64
Er_%                           object
Pr_%                           object
Her2_Ihq_+++                   object
Fish/Sish                      object
Ki67_%                         object
Grado                          object
dtype: object


##### Podemos identificar varios tipos de datos que no son correctos, por lo que...

- Convertiremos las fechas a "datetime".
- Los valores de Er_% y Pr_% deben ser convertidos a enteros.



In [116]:
# Cambiamos fechas a datetime

for col in luminalDf.columns:
    if 'Fecha' in col:
        luminalDf[col] = pd.to_datetime(luminalDf[col], errors='coerce') 

columnasFechas = [col for col in luminalDf.columns if 'Fecha' in col]


print(luminalDf[columnasFechas].dtypes)

Fecha_De_Nacimiento           datetime64[ns]
Fecha_De_Dx_Biopsia           datetime64[ns]
Fecha_1°_Consulta_Om_Cecan    datetime64[ns]
dtype: object


In [117]:
# Convertimos las columnas Er_%, Pr_%, Grado y Ki67_% a enteros, forzando errores a NaN y luego a Int64 para permitir nulos

luminalDf = luminalDf.copy()  
luminalDf['Er_%'] = pd.to_numeric(luminalDf['Er_%'], errors='coerce').astype('Int64')
luminalDf['Pr_%'] = pd.to_numeric(luminalDf['Pr_%'], errors='coerce').astype('Int64')
luminalDf['Grado'] = pd.to_numeric(luminalDf['Grado'], errors='coerce').astype('Int64')
luminalDf['Ki67_%'] = pd.to_numeric(luminalDf['Ki67_%'], errors='coerce').astype('Int64')


print(luminalDf[['Er_%', 'Pr_%', 'Grado', 'Ki67_%']].dtypes)



Er_%      Int64
Pr_%      Int64
Grado     Int64
Ki67_%    Int64
dtype: object


---
Una vez que corregimos los tipos de datos, verificamos que si estén correctos


In [118]:
print(luminalDf.dtypes)

Id                                    object
Fecha_De_Nacimiento           datetime64[ns]
Fecha_De_Dx_Biopsia           datetime64[ns]
Edad_Al_Dx_(Años)                      int64
Fecha_1°_Consulta_Om_Cecan    datetime64[ns]
Imc                                  float64
Er_%                                   Int64
Pr_%                                   Int64
Her2_Ihq_+++                          object
Fish/Sish                             object
Ki67_%                                 Int64
Grado                                  Int64
dtype: object


---
##### Buscamos datos únicos para cada columna 

Para observar irregularidades y presencia de valores duplicados de mejor manera.

In [119]:
# Mostramos los valores únicos de las columnas de interés para identificar irregularidades

try:
    for col in ['Er_%', 'Pr_%', 'Her2_/_Neu', 'Her2_Ihq_+++','Fish/Sish', 'Ki67_%', 'Grado']:
        if col in luminalDf.columns:
            print(f'Columna: {col}\nValores únicos:\n{luminalDf[col].unique()}\n')
        else:
            print(f'Columna {col} no encontrada en luminalDf\n')
            
except Exception as e:
    pass


Columna: Er_%
Valores únicos:
<IntegerArray>
[  10, <NA>,    0,   80,  100,   15,   20,   90,    2,   85,   70,   12,   98,
   60,    5,   95,   30,   50,    4,   65,   40,    7,    1,   55,    8,    6,
   75,    3,   91,   25,   96,   35,   99,    9,   16]
Length: 35, dtype: Int64

Columna: Pr_%
Valores únicos:
<IntegerArray>
[ 100,    5, <NA>,   60,   90,    0,   95,   40,   80,    2,   20,   70,   10,
    3,   50,   25,   30,    8,    7,   35,   65,   75,   15,    1,   18,   22,
   12,    4,   85,   99,   98,    6,   88,   92,   68,    9]
Length: 36, dtype: Int64

Columna Her2_/_Neu no encontrada en luminalDf

Columna: Her2_Ihq_+++
Valores únicos:
['1+' 'No aplica' '0+' '2+' '3+' 'Desconocido']

Columna: Fish/Sish
Valores únicos:
['No aplica' 'Negativo' 'Positivo' 'Desconocido']

Columna: Ki67_%
Valores únicos:
<IntegerArray>
[   5, <NA>,   20,    0,   90,   10,   50,   40,   60,    8,   65,   80,   95,
   25,   30,   70,   15,   45,    3,   12,   47,   14,    7,   35,    2,    1,
 

##### Observamos datos redundantes cómo:

- La misma palabra con espacio(s) adicional(es).
- Datos que no podemos inferir por lo que tendrían que ser descartados (e.g. "No Aplica", "Desconocido")

---
##### Hacemos la limpieza de los datos, esto incluye:

- Reemplazar los valores que no pueden ser inferidos por NaN (aún no serán eliminados debido a su dependencia a otras columnas que si necesitamos).
- Quitar espacios extra en todos los valores para evitar la duplicidad de datos (e.g. "   Negativo ", "Negativo").
- Convertir todos los datos tipo string a minúsculas para evitar problemas de capitalización adecuada.
- Convertir strings a int cuando se pueda hacer (e.g. "1+, 2+, 3+" => 1,2,3)



In [120]:

import numpy as np

# Las columnas categóricas que vamos a limpiar
categorical_cols = ['Her2_Ihq_+++', 'Fish/Sish', 'Grado']  

# Convertir a lowercase para evitar problemas de capitalización

for col in categorical_cols:
    if luminalDf[col].dtype == 'object':  
        luminalDf[col] = luminalDf[col].str.lower()

# Eliminar datos duplicados en las columnas categóricas, usar strip para verificar duplicidad

luminalDf['Her2_Ihq_+++'] = luminalDf['Her2_Ihq_+++'].str.strip()
luminalDf['Fish/Sish'] = luminalDf['Fish/Sish'].str.strip()


# Reemplazar 'No Aplica' y 'Desconocido' por np.nan en las columnas categóricas

willReplaceValues = ['no aplica', 'desconocido']

for col in categorical_cols:

    luminalDf[col] = luminalDf[col].replace(willReplaceValues, np.nan)


# Convertir los valores tipo "1+", "2+", "3+", "0+" a enteros, pero primero manejar nulos

luminalDf['Her2_Ihq_+++'] = luminalDf['Her2_Ihq_+++'].map({'0+': 0, '1+': 1, '2+': 2, '3+': 3}) 
luminalDf['Her2_Ihq_+++'] = luminalDf['Her2_Ihq_+++'].infer_objects(copy=False)

#replace nulos con la media redondeada

luminalDf['Her2_Ihq_+++'] = luminalDf['Her2_Ihq_+++'].fillna(luminalDf['Her2_Ihq_+++'].mean().round()).astype(int)


Revisamos valores únicos en las columnas nuevamente después de la limpieza


In [121]:
#Revisamos valores únicos en las columnas nuevamente después de la limpieza

for col in ['Er_%', 'Pr_%', 'Her2_/_Neu', 'Her2_Ihq_+++','Fish/Sish', 'Ki67_%', 'Grado']:
    if col in luminalDf.columns:
        print(f'Columna: {col}\nValores únicos:\n{luminalDf[col].unique()}\n')

    else:
        print(f'Columna {col} no encontrada en luminalDf\n')


Columna: Er_%
Valores únicos:
<IntegerArray>
[  10, <NA>,    0,   80,  100,   15,   20,   90,    2,   85,   70,   12,   98,
   60,    5,   95,   30,   50,    4,   65,   40,    7,    1,   55,    8,    6,
   75,    3,   91,   25,   96,   35,   99,    9,   16]
Length: 35, dtype: Int64

Columna: Pr_%
Valores únicos:
<IntegerArray>
[ 100,    5, <NA>,   60,   90,    0,   95,   40,   80,    2,   20,   70,   10,
    3,   50,   25,   30,    8,    7,   35,   65,   75,   15,    1,   18,   22,
   12,    4,   85,   99,   98,    6,   88,   92,   68,    9]
Length: 36, dtype: Int64

Columna Her2_/_Neu no encontrada en luminalDf

Columna: Her2_Ihq_+++
Valores únicos:
[1 0 2 3]

Columna: Fish/Sish
Valores únicos:
[nan 'negativo' 'positivo']

Columna: Ki67_%
Valores únicos:
<IntegerArray>
[   5, <NA>,   20,    0,   90,   10,   50,   40,   60,    8,   65,   80,   95,
   25,   30,   70,   15,   45,    3,   12,   47,   14,    7,   35,    2,    1,
   75,   87,   18,  100,   24,   55,    4,   22,   67,   72, 

In [122]:
print(luminalDf.isna().sum())  # Nulos por columna
print(luminalDf.isna().sum(axis=1).value_counts()) 


print("Cantidad de registros:", luminalDf.shape[0])


Id                               0
Fecha_De_Nacimiento              1
Fecha_De_Dx_Biopsia              1
Edad_Al_Dx_(Años)                0
Fecha_1°_Consulta_Om_Cecan       3
Imc                              0
Er_%                            30
Pr_%                            31
Her2_Ihq_+++                     0
Fish/Sish                     1253
Ki67_%                         501
Grado                           15
dtype: int64
1    819
2    438
0     61
4     15
5     13
3      5
Name: count, dtype: int64
Cantidad de registros: 1351


---
### Para la siguiente parte consideramos lo siguiente...

##### Criterios para catalogar luminales

- Si el porcentaje de estrógeno (Er_%) > 50, el Receptor E es Triple Positivo (+++).
- Si el porcentaje de estrógeno (Er_%) <= 50 y > 0, el Receptor E es Doble Positivo (++).

- Si el porcentaje de progesterona (Pr_%) > 20, el Receptor Pr es Triple Positivo (+++).
- Si Her2_Ihq+++ es "0+" o "1+", el tumor es de HER2 Negativo.
- Si Her2_Ihq+++ es "2+", se requiere Fish/Sish para confirmar (y tendría que ser un Fish/Sish Negativo para que el HER2 sea Negativo).






---
Reemplazamos valores nulos de columnas con pocos valores faltantes con la media


In [123]:
# Nos deshacemos de los registros que no pueden ser luminales

# Reemplazamos nulos en Er_%, Pr_%, Grado, Her2_Ihq_+++ con la media para facilitar el filtrado, esto con la
# justificacion que son pocos registros (<50) y no afectará el análisis

luminalDf['Er_%'] = luminalDf['Er_%'].fillna(int(luminalDf['Er_%'].mean()))

luminalDf['Pr_%'] = luminalDf['Pr_%'].fillna(int(luminalDf['Pr_%'].mean()))
luminalDf['Grado'] = luminalDf['Grado'].fillna(int(luminalDf['Grado'].mean()))



#reemplazar fechas nulas con la fecha más común (moda)
luminalDf['Fecha_De_Nacimiento'] = luminalDf['Fecha_De_Nacimiento'].fillna(luminalDf['Fecha_De_Nacimiento'].mode()[0])
luminalDf['Fecha_De_Dx_Biopsia'] = luminalDf['Fecha_De_Dx_Biopsia'].fillna(luminalDf['Fecha_De_Dx_Biopsia'].mode()[0])
luminalDf['Fecha_1°_Consulta_Om_Cecan'] = luminalDf['Fecha_1°_Consulta_Om_Cecan'].fillna(luminalDf['Fecha_1°_Consulta_Om_Cecan'].mode()[0])

Nos aseguramos que ya llenamos los nulos que pueden ser reemplazados.

In [124]:
print(luminalDf.isna().sum()) 
print(luminalDf.isna().sum(axis=1).value_counts()) 


Id                               0
Fecha_De_Nacimiento              0
Fecha_De_Dx_Biopsia              0
Edad_Al_Dx_(Años)                0
Fecha_1°_Consulta_Om_Cecan       0
Imc                              0
Er_%                             0
Pr_%                             0
Her2_Ihq_+++                     0
Fish/Sish                     1253
Ki67_%                         501
Grado                            0
dtype: int64
1    826
2    464
0     61
Name: count, dtype: int64


##### Buscamos los datos que ya nos permiten generar una respuesta y creamos las columnas correspondientes.

In [125]:
# Para determinar nuestro Receptor Pr, sabemos que...

luminalDf['Receptor_Pr'] = '-'
luminalDf.loc[luminalDf['Pr_%'] > 20, 'Receptor_Pr'] = '+++'
luminalDf.loc[(luminalDf['Pr_%'] <= 20) & (luminalDf['Pr_%'] > 0), 'Receptor_Pr'] = '+/-' # sabemos que esto es + o - 

#Confirmamos el reemplazo adecuado
print(luminalDf['Receptor_Pr'].unique()) 

['+++' '+/-' '-']


In [126]:
#Mostramos la cantidad de registros después de filtrar Er_%

print("Cantidad de registros:", luminalDf.shape[0])

Cantidad de registros: 1351


Filtramos los que sabemos que no pueden ser luminales por porcentaje de Ki67

In [127]:
# Sabemos que un Ki67 > 30% es alto, cuando es alto entonces no aplica para los luminales A o B


luminalDf = luminalDf[luminalDf['Ki67_%'] <= 30]


In [128]:
print("Cantidad de registros:", luminalDf['Ki67_%'].shape[0])

Cantidad de registros: 549


Sabemos que el Receptor E debe ser triple o doble positivo para catalogar como luminal.

In [129]:
# Un Er_% mayor a 0 es requisito para ser luminal A o B

luminalDf = luminalDf[luminalDf['Er_%'] > 0]

# Un Er_% mayor a 50% es triple positivo, si es menor o igual a 50% es doble positivo

luminalDf['Receptor_E'] = '+++'
luminalDf.loc[luminalDf['Er_%'] <= 50, 'Receptor_E'] = '++'

print(luminalDf['Receptor_E'].unique()) # Confirmamos el reemplazo adecuado


['++' '+++']


Sabemos que si el Her2_Ihq+++ es 0+ o 1+ entonces es HER2 negativo, por lo que nos deshacemos del 3+

In [130]:
# Sabemos que si el Her2_Ihq+++ es 0+ o 1+ entonces es HER2 negativo, por lo que nos deshacemos del 3+

luminalDf = luminalDf[luminalDf['Her2_Ihq_+++'] <= 2]

In [131]:
print("Cantidad de registros:", luminalDf.shape[0])

Cantidad de registros: 389


Si el Her2_Ihq+++ es 2+, sabemos que para ser un HER2 negativo se necesita el Fish/Sish negativo

In [132]:
# Si el Her2_Ihq+++ es 2+, sabemos que para ser un HER2 negativo se necesita el Fish/Sish negativo

luminalDf = luminalDf[~((luminalDf['Her2_Ihq_+++'] == 2) & (luminalDf['Fish/Sish'] != 'negativo'))]

# vemos que ahora en todas las filas, fish es nulo

print(luminalDf[(luminalDf['Fish/Sish'] == 'negativo')][['Her2_Ihq_+++']].sum()) 

print(luminalDf['Fish/Sish'].isna().sum()) 


Her2_Ihq_+++    24
dtype: int64
367


In [133]:
print("Cantidad de registros:", luminalDf.shape[0])

Cantidad de registros: 379


##### Criterios para catalogar diferenciar Luminales Tipo A y B

- Si el Receptor Pr es Triple Negativo el tumor es Luminal A, si es Positivo o Negativo podría ser Luminal B.

- Si el Grado es 1 el tumor es Luminal A, si es 2 entonces podría tanto A cómo B, si es 3 podría ser Luminal B o cualquier otro fuera de nuestro enfoque.

- Si el porcentaje de Ki67 es menor del 14%, el tumor es Luminal A, si está entre 14% - 30%, podría ser Luminal B, sin embargo también podría ser fuera de nuestro enfoque.




In [ ]:
def catalogar_luminal(row):
    
    if row['Receptor_Pr'] == '+++' and row['Grado'] == 1 and row['Ki67_%'] < 14:
        return 'Luminal A'
    
    elif (row['Receptor_Pr'] in ['+/-', '-'] or row['Grado'] == 2 or (row['Ki67_%'] >= 14 and row['Ki67_%'] <= 30)):
        return 'Luminal B'
    
    else:
        return 'No Luminal'

luminalDf['Tipo_Luminal'] = luminalDf.apply(catalogar_luminal, axis=1)

print(luminalDf['Tipo_Luminal'].value_counts()) #TODO acabar doc analisis de nuevas columnbas y documentacion

Tipo_Luminal
Luminal B     306
Luminal A      47
No Luminal     26
Name: count, dtype: int64
